Deben crear 6 funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

In [ ]:
import pandas as pd 
import numpy as np
import ast
from fastapi import FastAPI
app = FastAPI()
from collections import defaultdict
import calendar
import asyncio
import json 


In [ ]:
#cargamos la data
data = pd.read_csv("mi_movies_dataset.csv")

In [ ]:
data.head( )

In [ ]:
#cambiamos el tipo de pada de pais 
data["production_companies"] = data["production_companies"].astype(str)
data["production_companies"].fillna("Sin datos")

In [ ]:
data["production_countries"] = data["production_countries"].astype(str)

In [ ]:
#desanidamos los elemnetos de pais
data["production_countries"] = data["production_countries"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
data.fillna("", inplace=True)

In [124]:
data["budget"].replace("", inplace=True)
data["revenue"].replace("", inplace=True)

In [125]:
data['budget'] = data['budget'].astype(float)

In [126]:
data['revenue'] = data['revenue'].astype(float)

In [119]:
#convierto los valores de la columna las siguientes columnas en valores flotantes para poder realizar la duvision ya que estaban como objetos
def calcular_retorno(row):
    revenue = row['revenue']
    budget = row['budget']
    
    if pd.notnull(revenue) and pd.notnull(budget) and budget != 0:
        return revenue / budget
    else:
        return 0

In [127]:
data['return'] = data.apply(calcular_retorno, axis=1)

In [ ]:
def peliculas_mes(mes: str): 
    '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron 
        ese mes 
    ''' 
    #Convertimos a la columna de tipo datetime
    data["release_date"] = pd.to_datetime(data["release_date"])
    
    #Obtenemos mes y año de la columna
    data['mes'] = data['release_date'].dt.month_name() 
    data['año'] = data['release_date'].dt.year

    #Filtramos por el mes proporcionado
    peliculas_mes = data[data['mes'].str.lower() == mes.lower()]
    
    #Cantidada de pelicula por mes 
    cantidad = len(peliculas_mes)
    
    return {'mes':mes, 'cantidad':cantidad}

In [ ]:
x= peliculas_mes("October")
print(x)

In [ ]:
def peliculas_dia(dia:str):
    '''
    Se ingresa el dia y la funcion retorna la cantidad de peliculas que se 
    estrenaron ese dia 

    ''' 
    # Convertir la columna a tipo datetime 
    data['release_date'] = pd.to_datetime(data['release_date']) 

    # Obtener el día de la semana de la columna
    data['dia_semana'] = data['release_date'].dt.day_name()

    # Filtrar  por el día de la semana proporcionado
    peliculas_dia = data[data['dia_semana'].str.lower() == dia.lower()]
 
    cantidad = len(peliculas_dia) 
    return  {'dia': dia, 'cantidad': cantidad}

In [ ]:
x = peliculas_dia("Monday")
print (x)

In [ ]:
def franquicia(franquicia): 
    '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y 
    promedio
    ''' 
    # Filtrar el DataFrame por la franquicia proporcionada
    franquicia_data = data[data['belongs_to_collection'].notnull() & data['belongs_to_collection'].apply(lambda x: isinstance(x, str) and franquicia.lower() in x.lower())]

    # Obtener la cantidad de películas en la franquicia
    cantidad = len(franquicia_data)

    # Calcular la ganancia total y el promedio de la franquicia
    ganancia_total = franquicia_data['revenue'].sum()
    ganancia_promedio = franquicia_data['revenue'].mean()

    
    return {'franquicia':franquicia, 'cantidad': cantidad, 'ganancia_total':ganancia_total, 'ganancia_promedio':ganancia_promedio}

In [ ]:
respuesta = franquicia('Father of the Bride Collection')
print(respuesta)

In [ ]:
def peliculas_pais(pais: str):
    
    # Filtrar la data por el país proporcionado
    pelicula_pais = data[data["production_countries"].str.contains(pais, case=False, na=False)]


    # cantidad de películas producidas en el país
    cantidad = len(pelicula_pais)

    return {'pais': pais, 'cantidad': cantidad}

In [ ]:
x = peliculas_pais("United States of America")
print(x)

In [ ]:
def productoras(productora: str):
    '''
    Ingresas la productora, retornando la ganancia total y 
    la cantidad de peliculas que produjeron
    '''
    # Filtrar el DataFrame por la productora proporcionada
    productora_filtro = data[data['production_companies'].notnull() & data['production_companies'].apply(lambda x: any(p['name'].lower() == productora.lower() for p in ast.literal_eval(x)))]

    # Obtener la ganancia total y la cantidad de películas producidas por la productora
    ganancia_total = productora_filtro['revenue'].sum()-productora_filtro['budget'].sum()
    cantidad = len(productora_filtro)

    return {'productora': productora, 'ganancia_total': ganancia_total, 'cantidad': cantidad}

In [128]:
def productoras1(productora:str):
    
    produ = data[["production_companies", "budget", "revenue"]].dropna()
    produ["production_companies"] = produ["production_companies"].map(str.lower)

    cantidad = produ.shape[0]
    ganacia_total = (produ["revenue"]- produ["budget"]).sum()

    return {"productora": productora, "ganancia total": ganacia_total, "cantidad": cantidad}

In [129]:
x = productoras1("Interscope Communications")
print(x)

{'productora': 'Interscope Communications', 'ganancia total': 317515442702.0, 'cantidad': 45453}


In [130]:
def retorno(pelicula: str): 
    '''Ingresas la pelicula, retornando la inversion, la ganancia, 
    el retorno y el año en el que se lanzo
    ''' 
    #filtramos las filas correspondientes 
    fila_pelicula = data[data["title"] == pelicula].iloc[0]
    
    #sacamos los valores 
    inversion = float(fila_pelicula["budget"])
    ganancia = float(fila_pelicula["revenue"]) - inversion
    retorno = float(fila_pelicula["return"])
    anio = int(fila_pelicula["release_year"])

    return {'pelicula':pelicula, 'inversion':inversion, 'ganacia':ganancia,'retorno': retorno, 'anio': anio}

In [131]:
x = retorno("Jumanji")
print(x)

{'pelicula': 'Jumanji', 'inversion': 65000000.0, 'ganacia': 197797249.0, 'retorno': 4.0430346, 'anio': 1995}
